In [1]:
from container_component_src.model.datamodule import TimeStampDataset
from container_component_src.model.lightning_module import TimeStampVAE
from torch.utils.data import DataLoader
import pandas as pd
import os

In [2]:
test_df_path = 's3://mlpipeline-henrik-sebastian-steude/v2/artifacts/columbus-eclss-ad-pipeline/40d6cd03-6cf7-4272-918a-bff69869d3a7/create-train-dev-test-split/3b7bedd7-61c2-4dbf-a253-5965be66632a/df_test'
storage_options = {
    "key": os.environ["AWS_ACCESS_KEY_ID"],
    "secret": os.environ["AWS_SECRET_ACCESS_KEY"],
    "client_kwargs": {"endpoint_url": f'http://{os.environ["S3_ENDPOINT"]}'},
}
test_df = pd.read_parquet(test_df_path, storage_options=storage_options)

In [3]:
test_dl =  DataLoader(
            TimeStampDataset(test_df),
            batch_size=10,
            num_workers=1,
)
batch = next(iter(test_dl))

In [4]:
batch.shape

torch.Size([10, 182])

In [5]:
batch.numpy().shape

(10, 182)

In [13]:
import requests
import json
import numpy as np

api_key = "xxxxx" # Change me!

# Create sample data with the CORRECT dimensions
batch_size = 10
expected_features = 181  # Model expects exactly 181 features

response_list = []
for batch in test_dl:
    # Your current data has 182 features, so we'll trim it
    current_data = batch.numpy()
    trimmed_data = current_data[:, :expected_features]  # Keep only the first 181 features
    
    # Convert to the proper format for the request
    batch_data = trimmed_data.astype(np.float32).tolist()
    
    # Print shape for verification
    # print(f"Adjusted input shape: {np.array(batch_data).shape}")
    
    # Send the request with the correct shape
    response = requests.post(
        "http://eclss-point-vae.henrik-sebastian-steude.svc.cluster.local/v2/models/eclss-vae/infer",
        headers={"Content-Type": "application/json", "X-API-Key": api_key},
        data=json.dumps({
            "inputs": [{
                "name": "input__0",
                "shape": [batch_size, expected_features],  # Explicitly set to [10, 181]
                "datatype": "FP32",
                "data": batch_data
            }]
        })
    )
    response_list.append(response.content)


In [14]:
response_list[0]

b'{"model_name":"eclss-vae","model_version":"1","outputs":[{"name":"output__0","datatype":"FP32","shape":[10],"data":[182143920.0,182921104.0,181859888.0,181804672.0,182453760.0,182667856.0,182199664.0,181941872.0,182357408.0,182331344.0]}]}'

In [15]:
len(response_list)

3741